# Régularisation avec SciKit-Learn

Auparavant, nous avons créé un nouvel ensemble de features / caractéristiques polynomiales, puis nous lui avons appliqué une régression linéaire standard, mais nous pouvons être plus intelligents dans le choix du modèle et utiliser la régularisation.

La régularisation tente de minimiser la RSS (somme résiduelle des carrés) *et* un facteur de pénalité. Ce facteur de pénalité pénalisera les modèles dont les coefficients sont trop importants. Certaines méthodes de régularisation font en sorte que des features non utiles aient un coefficient de zéro, auquel cas le modèle ne tient pas compte de la feature.

Nous allons explorer deux méthodes de régularisation, la régression de Ridge et la régularisation du Lasso. Nous les combinerons avec l'ensemble de features polynomiales (il ne serait pas aussi efficace d'effectuer la régularisation d'un modèle sur un ensemble de features d'origine aussi petit que celui de l'X d'origine).

## Importations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data et Setup

In [ ]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

### Conversion Polynomiale

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [ ]:
poly_features = polynomial_converter.fit_transform(X)

### Train | Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

## Mise à l'échelle des Données - Scaling Data

Bien que notre ensemble de données particulier ait toutes les valeurs dans le même ordre de grandeur (des milliers de dollars dépensés), ce n'est généralement pas le cas sur un ensemble de données, et puisque les mathématiques derrière les modèles régularisés additionnent les coefficients, il est important de normaliser les featues. Consultez les vidéos théoriques pour plus d'informations, ainsi que la discussion sur la raison pour laquelle nous n'effectuons que **fit** aux données d'entraînement et **transform** sur les deux ensembles séparément.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# help(StandardScaler)

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X_train)

StandardScaler()

In [ ]:
X_train = scaler.transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

## Régression Ridge

Assurez-vous de visionner les leçons en vidéo pour une explication complète de la régression de Ridge et du choix d'un alpha.

In [ ]:
*from sklearn.linear_model import Ridge

In [ ]:
ridge_model = Ridge(alpha=10)

In [ ]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [ ]:
test_predictions = ridge_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

0.5774404204714162

In [ ]:
RMSE

0.8946386461319651

Comment s'est-il comporté sur l'ensemble d'entraînement ? (Cet ensemble sera utilisé ultérieurement à des fins de comparaison).

In [ ]:
# Performance sur Training Set
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.528834818302531

### Choisir une valeur alpha avec la validation croisée (cross-validation)

Consultez la vidéo sur la théorie pour obtenir tous les détails.

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
# help(RidgeCV)

In [ ]:
# Chooisir un scoring : https://scikit-learn.org/stable/modules/model_evaluation.html
# RMSE négatif, donc toutes les metrics suivent la convention "Plus c'est haut, mieux c'est".

# Voir toutes les options : sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [ ]:
# Plus vous passez d'options alpha, plus cela prendra du temps.
# Heureusement, notre ensemble de données est encore assez petit.
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [ ]:
ridge_cv_model.alpha_

0.1

In [ ]:
test_predictions = ridge_cv_model.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

0.4273774884345439

In [ ]:
RMSE

0.6180719926946001

In [ ]:
# Performance sur Training Set
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.30941321056483123

In [ ]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

## Régression Lasso

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [ ]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [ ]:
lasso_cv_model.alpha_

0.4943070909225832

In [ ]:
test_predictions = lasso_cv_model.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

0.6541723161252867

In [ ]:
RMSE

1.1308001022762548

In [ ]:
# Performance sur Training Set
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.6912807140820709

In [ ]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

Elastic Net combine les pénalités de la régression de Ridge et de la régression du Lasso dans le but d'obtenir le meilleur des deux mondes !

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [ ]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [ ]:
elastic_model.l1_ratio_

1.0

In [ ]:
test_predictions = elastic_model.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

0.5663262117569452

In [ ]:
RMSE

0.7485546215633726

In [ ]:
# Performance sur Training Set
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.43075829904723684

In [ ]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])